In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re

import projects_parser

In [3]:
BOOKLET_FILE = "../booklets_text/2013.txt"

In [4]:
with open(BOOKLET_FILE, encoding ="utf-8") as f:
    lines = f.readlines()

In [5]:
print(lines[0:20])
print(len(lines))

['Project Title: PyPlay\n', 'Name: Gautam Wadhwa\n', 'Email: gautam.wadhwa2@mail.dcu.ie\n', 'Programme: Software Engineering\n', 'Supervisor: Brian Stone\n', 'The title of this project is ‘PyPlay’. It is an MP3 player that can both play local MP3, OGG and FLAC\n', 'files and MP3s streamed from a server. Additional features of the project include MP3 metadata\n', 'display and support for opening and creating music playlists. The project will also include a server to\n', 'stream the MP3s. This project comes from passion for music and recognising that streaming MP3\n', 'files from a server would be both convenient and useful. Many people like to listen to music but do not\n', 'want to keep separate copies of the same music library on different machines. Other people like to\n', 'listen to music, but they travel a lot and cannot be near their music library all the time. This project\n', 'aims to solve both of these problems by letting users stream their music collections from a server.\n',

In [6]:
# get rid of project numbering
filter_pattern = re.compile(r'^Project Number', re.M)
lines_filtered = list(filter(lambda line: not filter_pattern.match(line), lines))

In [7]:
print(lines_filtered[0:30])
print(len(lines_filtered))

['Project Title: PyPlay\n', 'Name: Gautam Wadhwa\n', 'Email: gautam.wadhwa2@mail.dcu.ie\n', 'Programme: Software Engineering\n', 'Supervisor: Brian Stone\n', 'The title of this project is ‘PyPlay’. It is an MP3 player that can both play local MP3, OGG and FLAC\n', 'files and MP3s streamed from a server. Additional features of the project include MP3 metadata\n', 'display and support for opening and creating music playlists. The project will also include a server to\n', 'stream the MP3s. This project comes from passion for music and recognising that streaming MP3\n', 'files from a server would be both convenient and useful. Many people like to listen to music but do not\n', 'want to keep separate copies of the same music library on different machines. Other people like to\n', 'listen to music, but they travel a lot and cannot be near their music library all the time. This project\n', 'aims to solve both of these problems by letting users stream their music collections from a server.\n',

In [8]:
# Separate lines into list of projects

PROJECT_SEPARATOR = 'Project Title'
project_title_indices = [i for i, line in enumerate(lines_filtered) if PROJECT_SEPARATOR in line]
print(project_title_indices[0:5])

projects_strings = []
i = 1
while i < len(project_title_indices):
    projects_strings.append("".join(lines_filtered[project_title_indices[i-1]:project_title_indices[i]]))
    i += 1
projects_strings.append("".join(lines_filtered[project_title_indices[-1]:]))

[0, 16, 32, 62, 84]


In [9]:
print(projects_strings[0:3])
print(len(projects_strings))

['Project Title: PyPlay\nName: Gautam Wadhwa\nEmail: gautam.wadhwa2@mail.dcu.ie\nProgramme: Software Engineering\nSupervisor: Brian Stone\nThe title of this project is ‘PyPlay’. It is an MP3 player that can both play local MP3, OGG and FLAC\nfiles and MP3s streamed from a server. Additional features of the project include MP3 metadata\ndisplay and support for opening and creating music playlists. The project will also include a server to\nstream the MP3s. This project comes from passion for music and recognising that streaming MP3\nfiles from a server would be both convenient and useful. Many people like to listen to music but do not\nwant to keep separate copies of the same music library on different machines. Other people like to\nlisten to music, but they travel a lot and cannot be near their music library all the time. This project\naims to solve both of these problems by letting users stream their music collections from a server.\nPrimary Area: Multimedia\nPrimary OS: Unix/Linux\n

In [10]:
regex_schema = {
    'title': r'Project Title:(.*?)Name',
    'students': r'Name:(.*?)Programme',
    'programme': r'Programme:(.*)Supervisor',
    'supervisor_and_description': r'Supervisor:(.*?)(?:Primary Area:|Secondary Area:|Primary OS:|Primary Technology:|Secondary Technology:|$)',
    'primary_area': r'Primary Area:(.*?)(?:Secondary Area:|Primary OS:|Primary Technology:|Secondary Technology:|$)',
    'secondary_area': r'Secondary Area:(.*?)(?:Primary OS:|Primary Technology:|Secondary Technology:|$)',
    'primary_os': r'Primary OS:(.*?)(?:Primary Technology:|Secondary Technology:|$)',
    'primary_tech': r'Primary Technology:(.*?)(?:Secondary Technology|$)',
    'secondary_tech': r'Secondary Technology:(.*?)$',
}

In [11]:
projects = projects_parser.parse_projects(projects_strings, regex_schema)

secondary_area ANOMALY: 0 Project Title: PyPlay
Name: Gautam Wadhwa
Email: gautam.wadhwa2@mail.dcu.ie
Programme: Software Engineering
Supervisor: Brian Stone
The title of this project is ‘PyPlay’. It is an MP3 player that can both play local MP3, OGG and FLAC
files and MP3s streamed from a server. Additional features of the project include MP3 metadata
display and support for opening and creating music playlists. The project will also include a server to
stream the MP3s. This project comes from passion for music and recognising that streaming MP3
files from a server would be both convenient and useful. Many people like to listen to music but do not
want to keep separate copies of the same music library on different machines. Other people like to
listen to music, but they travel a lot and cannot be near their music library all the time. This project
aims to solve both of these problems by letting users stream their music collections from a server.
Primary Area: Multimedia
Primary OS: Un

In [12]:
print(projects[0:3])

[{'title': ' PyPlay\n', 'students': ' Gautam Wadhwa\nEmail: gautam.wadhwa2@mail.dcu.ie\n', 'programme': ' Software Engineering\n', 'supervisor_and_description': ' Brian Stone\nThe title of this project is ‘PyPlay’. It is an MP3 player that can both play local MP3, OGG and FLAC\nfiles and MP3s streamed from a server. Additional features of the project include MP3 metadata\ndisplay and support for opening and creating music playlists. The project will also include a server to\nstream the MP3s. This project comes from passion for music and recognising that streaming MP3\nfiles from a server would be both convenient and useful. Many people like to listen to music but do not\nwant to keep separate copies of the same music library on different machines. Other people like to\nlisten to music, but they travel a lot and cannot be near their music library all the time. This project\naims to solve both of these problems by letting users stream their music collections from a server.\n', 'primary_a

In [13]:
projects_final = projects_parser.canonicalize_projects(projects, name_sep='Name:', email_sep='Email:')

In [14]:
projects_final[0:3]
len(projects_final)

106

In [15]:
projects_area_combined = projects_parser.apply_transformation(projects_final, 
    projects_parser.combine_fields,
    'area',
    'primary_area', 
    'secondary_area')
projects_tech_combined = projects_parser.apply_transformation(projects_area_combined,
    projects_parser.combine_fields,
    'technology',
    'primary_tech',
    'secondary_tech')

In [16]:
# For consistency with 2014, rename the primary_os field to platform
projects_platform = projects_parser.apply_transformation(projects_tech_combined, projects_parser.rename_field, 'platform', 'primary_os')

In [17]:
print(projects_platform[0:3])
print(len(projects_platform))

[{'title': 'PyPlay', 'students': [{'name': 'Gautam Wadhwa', 'email': 'gautam.wadhwa2@mail.dcu.ie'}], 'programme': 'Software Engineering', 'supervisor': 'Brian Stone', 'description': 'The title of this project is ‘PyPlay’. It is an MP3 player that can both play local MP3, OGG and FLAC files and MP3s streamed from a server. Additional features of the project include MP3 metadata display and support for opening and creating music playlists. The project will also include a server to stream the MP3s. This project comes from passion for music and recognising that streaming MP3 files from a server would be both convenient and useful. Many people like to listen to music but do not want to keep separate copies of the same music library on different machines. Other people like to listen to music, but they travel a lot and cannot be near their music library all the time. This project aims to solve both of these problems by letting users stream their music collections from a server.', 'area': 'Mul

In [18]:
projects_parser.write_json('2013', projects_platform)